In [23]:
import os
import sys

import numpy as np
import io
import time

from Database import *

import pandas as pd

os.environ['dropboxPath'] = '/home/mrware/ANAPC/online/Dropbox'
import matplotlib.pyplot as plt
sys.path.insert(0, os.environ['dropboxPath'] + '/Code/mattsTools')
from plotStyles import *
from picklez import *

In [24]:

import math
import os
import shutil
import os.path 
import time
from subprocess import* 

In [38]:
bl=3
#etag = getNewestTag( 'xfel_bl_3_tc_bm_2_pd/charge' )
#ftag=ftag-10000
#ftag = etag-100
hightag = getNewestHighTag( bl )

ftag=700803692
etag=700841344


In [39]:
print(ftag,etag,hightag)

700803692 700841344 201802


In [40]:
tmaid = 113

status = Popen("OutputTMACsvFromUDB -tmaid %d -hightag %d -starttag %d -endtag %d -out ./timetoolpixeltmp.csv"%(tmaid,hightag,ftag,etag),
                shell = True,stdout = PIPE)

In [41]:
stdout_value = status.communicate()
print (stdout_value)

(b'[OutputTMACsvFromUDB]processing...\n[OutputTMACsvFromUDB]Failed to tma_connect_udb. err=-10018.\n[OutputTMACsvFromUDB]failed.\n', None)


In [ ]:
OutputTMACsvFromUDB -tmaid 14 -hightag 201802 -starttag 700953270 -endtag 700953280 -out ./timetoolpixeltmp.csv

In [65]:

def pullTimeTool2Local( targetRun = None, bl=3,
                        sourceServer='mrware@xhpcfep.hpc.spring8.or.jp' , 
                        sourceDirectory='/home/mrware/TM_config/TM_output' ):


    commandString = 'ssh %s ls %s' % ( sourceServer , sourceDirectory )
    status = Popen(commandString, shell = True,stdout = PIPE)
    stdout_value = status.communicate()
    
    if targetRun is None:
        targetRun = getNewestRun( bl ) - 1
        
    if str(targetRun) in str(stdout_value[0]):
        scpString = 'scp %s:%s/%s.csv .' % ( sourceServer , sourceDirectory , str(targetRun) )
        status = Popen(scpString, shell = True,stdout = PIPE)
        stdout_value = status.communicate()
        time.sleep(1)
        df = pd.read_csv( str(targetRun)+'.csv' , skiprows=1 ).set_index('tag_number')
        del df.index.name
        os.remove( str(targetRun)+'.csv' )
        return df
    
    return None
        
df = pullTimeTool2Local(targetRun=551832)
df.head(10)



,timing_edge_derivative(pixel),timing_edge_fitting(pixel)
710248578,NaN,NaN
710248580,913.5,915.855384
710248582,NaN,NaN
710248584,NaN,NaN
710248586,NaN,NaN
710248588,950.5,950.672063
710248590,NaN,NaN
710248592,794.5,802.923877
710248594,768.5,765.612651
710248596,NaN,NaN


In [66]:
def addTT2Data( tagLow , tagHigh , ttDF , directory='/home/mrware/ANAPC/online/dataSaverLoader/smallData' ):
    lowTags = getLowTags( directory=directory )
    nearestIdx, nearestSaveTag = nearestValue( lowTags , tagLow )
    if nearestSaveTag > tagLow:
        if nearestIdx - 1 < 0:
            raise Exception( 'tagLow does not exist in saved data' )
        nearestSaveTag = lowTags[nearestIdx - 1]
        nearestIdx = nearestIdx - 1
        
    df = pd.read_pickle( directory+'/smallData_'+str(int(nearestSaveTag))+'.pkl' )
    df['tt'] = np.nan
    
    
    while df.index.values.max() < tagHigh:
        nearestIdx += 1
        if nearestIdx >= lowTags.shape[0]:
            break
        nearestSaveTag = lowTags[nearestIdx]
        df = pd.read_pickle( directory+'/smallData_'+str(int(nearestSaveTag))+'.pkl' )
        df['tt'] = np.nan


def adTT2RunData( bl , run , ttDF ,  directory='/home/mrware/ANAPC/online/dataSaverLoader/smallData'  ):
    tagLow, tagHigh = getTagRange( bl , run )
    return loadData( tagLow[1] , tagHigh[1] , ttDF , directory=directory )

KeyError: 'a'